## Imports

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.sparse.linalg import svds
from numpy.linalg import norm, svd

import itertools
import math
import re

from util.word2vec_as_MF import Word2vecMF
from util.functions import *

import time
dimension = 100
regularization =0.0
model_enwik = Word2vecMF()
negative = 5

Please run:
  python setup.py build_ext --inplace


## Read and preprocess enwik9

# Gensim

In [2]:
'''%%time
skip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'''

'%%time\nskip = Word2Vec(real_sentences, size = dimension, compute_loss=True)'

In [3]:
'''skip.get_latest_training_loss()'''

'skip.get_latest_training_loss()'

## Train ro_sgns model starting from SVD of SPPMI

In [4]:
%%time
# Create word2vobject has no attributeec as matrix factorization model

'''model_enwik.data_to_matrices(real_sentences, dimension, 5, 
                             DB_to_file='enwik-200/matrices.npz',
                            indices_to_file='enwik-200/vocab.txt')'''
#sentences = load('data/x1')

model_enwik.data_to_matrices('data/x1', r=dimension, k=negative, 
                             DB_to_file=False,
                            vocab_to_file=False)

Parsing sentences from training set
CPU times: user 837 ms, sys: 37.4 ms, total: 875 ms
Wall time: 883 ms


In [5]:
#model_enwik.load_matrices('data/x1.npz')


In [7]:
#C0, W0, step_size = BFGD_init(model_enwik, dimension=dimension, reg=regularization)
C0, W0 = SPPMI_init(model_enwik, dimension=dimension, negative = negative)
#C0, W0, step_size = RAND_init(model_enwik, dimension=dimension)
print(np.count_nonzero(C0))

0.047784370660270165
62.27460890120043
('Initial loss', 1808805.021334937)
35600


/home/yerong/local/Anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [8]:
print(C0.shape[1])
'''# Train the model
start_time = time.time()
opt_experiment(model_enwik,
               mode='AM', 
               d=dimension,
               eta = 5e-6,
               lbd = 1.0,
               MAX_ITER=189000,
               from_iter=0,
               start_from='SVD',
               init=(True, C_svd, W_svd), display=True)
print("--- %s seconds ---" % (time.time() - start_time))'''

356


'# Train the model\nstart_time = time.time()\nopt_experiment(model_enwik,\n               mode=\'AM\', \n               d=dimension,\n               eta = 5e-6,\n               lbd = 1.0,\n               MAX_ITER=189000,\n               from_iter=0,\n               start_from=\'SVD\',\n               init=(True, C_svd, W_svd), display=True)\nprint("--- %s seconds ---" % (time.time() - start_time))'

In [9]:
start_time = time.time()
opt_experiment(model_enwik,
               mode='PS',
               d=dimension,
               eta=1e-5,
               #eta = step_size,
               reg = regularization,
               MAX_ITER=5,
               from_iter=0,
               start_from='test',
               itv_print=1,
               itv_save=100000,
               init=(True, C0, W0), display=True,
               autostop=False)
print("--- %s seconds ---" % (time.time() - start_time))

enwik-200/PSiter_fromtest_dim100_step1e-05_0.0
('Iter #:', 0, 'loss', 1808805.021334937)
('Iter #:', 1, 'loss', 1808805.021334937)
('Iter #:', 2, 'loss', 1808568.2662150215)
('Iter #:', 3, 'loss', 1808339.1622854602)
('Iter #:', 4, 'loss', 1808116.5966503131)
('Iter #:', 5, 'loss', 1807899.4707686696)
--- 0.32144737243652344 seconds ---
